In [ ]:
#1. rutin1 import module
import pandas as pd
import os
import sys

In [ ]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

In [ ]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "data/dataset_ekstraksi_r29_lg.xlsx")
df = pd.read_excel(dataset_path)
df["text"] = df["judul"] +". "+ df["isi"]

In [ ]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

# Pre-trained Indobert model for Bahasa Indonesia
model_path = 'cahya/bert-base-indonesian-522M'
indobert = SentenceTransformer(model_path)

In [ ]:
df_keyphrases = pd.DataFrame(columns=['key_1', 'val_1', 'key_2', 'val_2', 'key_3', 'val_3'])
kw_model = KeyBERT(model=indobert)

for text in df["text"]:
    keyphrase = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3), top_n=3)

    # Separate the key and value elements into separate lists
    keys = [item[0] for item in keyphrase]
    values = [item[1] for item in keyphrase]

    # Create a dictionary with the keys and values
    data = {
    'key_1': [keys[0]],
    'val_1': [values[0]],
    'key_2': [keys[1]],
    'val_2': [values[1]],
    'key_3': [keys[2]],
    'val_3': [values[2]],
    }
    df_keyphrase = pd.DataFrame(data)
    
    # Concatenate df_keyphrase with df_keyphrases
    df_keyphrases = pd.concat([df_keyphrases, df_keyphrase], ignore_index=True)

In [ ]:
df_keyphrases

EVALUATION

In [ ]:
from utils import eval

targets = df[["k1", "k2", "k3","k4", "k5", "k6","k7"]].values.tolist()
df_targets = pd.DataFrame(targets)

In [ ]:
# Evaluation TextRank
predict_keybert_list = df_keyphrases[['key_1','key_2','key_3']].values.tolist()
eval_keybert = eval(predict_keybert_list, targets, True).round(3)
eval_keybert.columns = ['key_1', 'key_2','key_3','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_keybert = eval_keybert[['key_1', 'key_2','key_3', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
eval_keybert.head(3)

In [ ]:
# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
keybert_recall = eval_keybert['flex_recall'].mean()
keybert_prec = eval_keybert['flex_prec'].mean()
keybert_f1 = 2 * (keybert_prec * keybert_recall) / (keybert_prec + keybert_recall)

# Create a DataFrame with the scores
summary = pd.DataFrame({'textrank': [keybert_recall, keybert_prec, keybert_f1]}, index=['recall', 'precision', 'F1'])
summary = summary.round(3)
summary

In [ ]:
# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_keybert = pd.concat([df_keyphrases, df_targets, eval_keybert], axis=1)
predict_keybert.head(3)

In [ ]:
# Write predictions to excel file
from utils import write_excel

sheet_name = 'keybert'
output_file = 'std_keybert.xlsx'
write_excel(predict_keybert, sheet_name, output_file)

TODO
- membuat top-3 dengan keyBERT <done>
- membuat iterasi